In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [2]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [3]:
import tensorflow as tf

# Wide and Deep NN 1

In [4]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
ip=tf.keras.layers.Input(shape=X_train.shape[1:])
norm=tf.keras.layers.Normalization()
nlayer=norm(ip)
L=3
hlayers=[]
prev=nlayer
for i in range(L-1):
    hlayers.append(tf.keras.layers.Dense(30,activation='relu')(prev))
    prev=hlayers[i]
clayer=tf.keras.layers.Concatenate()([nlayer,prev])
olayer=tf.keras.layers.Dense(1)(clayer)
model=tf.keras.Model(inputs=[ip],outputs=[olayer])

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 8)]                  0         []                            
                                                                                                  
 normalization (Normalizati  (None, 8)                    17        ['input_1[0][0]']             
 on)                                                                                              
                                                                                                  
 dense (Dense)               (None, 30)                   270       ['normalization[0][0]']       
                                                                                                  
 dense_1 (Dense)             (None, 30)                   930       ['dense[0][0]']           

In [6]:
optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(loss='mse',optimizer=optimizer,metrics=['RootMeanSquaredError'])

In [7]:
norm.adapt(X_train)
history=model.fit(X_train,y_train,epochs=20,validation_data=(X_valid,y_valid))

Epoch 1/20


363/363 [==============================] - 4s 3ms/step - loss: 0.6538 - root_mean_squared_error: 0.8086 - val_loss: 0.7238 - val_root_mean_squared_error: 0.8508
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4045 - root_mean_squared_error: 0.6360 - val_loss: 4.7054 - val_root_mean_squared_error: 2.1692
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4170 - root_mean_squared_error: 0.6457 - val_loss: 0.5432 - val_root_mean_squared_error: 0.7370
Epoch 4/20
363/363 [==============================] - 1s 4ms/step - loss: 0.3894 - root_mean_squared_error: 0.6240 - val_loss: 1.7125 - val_root_mean_squared_error: 1.3086
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4854 - root_mean_squared_error: 0.6967 - val_loss: 12.8310 - val_root_mean_squared_error: 3.5820
Epoch 6/20
363/363 [==============================] - 1s 4ms/step - loss: 0.5910 - root_mean_squared_error: 0.7688 - val_loss: 16.1540 - 

In [8]:
mse_test=model.evaluate(X_test,y_test)

162/162 [==============================] - 0s 2ms/step - loss: 0.2976 - root_mean_squared_error: 0.5455


# Wide and Deep NN 2

In [10]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
ip_wide=tf.keras.layers.Input(shape=[5])# features 0 to 4
ip_deep=tf.keras.layers.Input(shape=[6])# features 2 to 7
norm1=tf.keras.layers.Normalization()
norm2=tf.keras.layers.Normalization()
nlayer_wide=norm1(ip_wide)
nlayer_deep=norm2(ip_deep)
L=3
hlayers=[]
prev=nlayer_deep
for i in range(L-1):
    hlayers.append(tf.keras.layers.Dense(30,activation='relu')(prev))
    prev=hlayers[i]
clayer=tf.keras.layers.Concatenate()([nlayer_wide,prev])
olayer=tf.keras.layers.Dense(1)(clayer)
model=tf.keras.Model(inputs=[ip_wide,ip_deep],outputs=[olayer])

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 6)]                  0         []                            
                                                                                                  
 normalization_1 (Normaliza  (None, 6)                    13        ['input_2[0][0]']             
 tion)                                                                                            
                                                                                                  
 input_1 (InputLayer)        [(None, 5)]                  0         []                            
                                                                                                  
 dense (Dense)               (None, 30)                   210       ['normalization_1[0][0]'] 

In [12]:

model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),metrics=['RootMeanSquaredError'])

In [13]:
X_train_wide,X_train_deep=X_train[:,:5],X_train[:,2:]
X_valid_wide,X_valid_deep=X_valid[:,:5],X_valid[:,2:]
X_test_wide,X_test_deep=X_test[:,:5],X_test[:,2:]
norm1.adapt(X_train_wide)
norm2.adapt(X_train_deep)
history=model.fit((X_train_wide,X_train_deep),y_train,epochs=20,validation_data=((X_valid_wide,X_valid_deep),y_valid))

Epoch 1/20
363/363 [==============================] - 2s 3ms/step - loss: 1.3737 - root_mean_squared_error: 1.1720 - val_loss: 1.0174 - val_root_mean_squared_error: 1.0087
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5221 - root_mean_squared_error: 0.7226 - val_loss: 0.7513 - val_root_mean_squared_error: 0.8668
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4350 - root_mean_squared_error: 0.6595 - val_loss: 0.3978 - val_root_mean_squared_error: 0.6307
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4017 - root_mean_squared_error: 0.6338 - val_loss: 0.3883 - val_root_mean_squared_error: 0.6231
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.3847 - root_mean_squared_error: 0.6203 - val_loss: 0.5088 - val_root_mean_squared_error: 0.7133
Epoch 6/20
363/363 [==============================] - 1s 3ms/step - loss: 0.3721 - root_mean_squared_error: 0.6100 - val_loss: 0.9047 - val_

In [14]:
mse_test=model.evaluate((X_test_wide,X_test_deep),y_test)

162/162 [==============================] - 0s 2ms/step - loss: 0.3174 - root_mean_squared_error: 0.5634
